# Step 1a MockUI: Setup an Information Request in KG
A user specifies their information request in natural language, this is processed much like an article.

![step1a](resources/Step1a_MockUI.png)

The result of this step includes:
- User nodes
- Request nodes, connected to User nodes with an REQUESTED_BY relationship
- Entity nodes, connected to Request nodes with a MENTIONED_IN relationship

In [ ]:
import os
import logging

#logging.getLogger("OpenTLDR").setLevel(logging.ERROR)  # Less output
#logging.getLogger("OpenTLDR").setLevel(logging.WARN)   # Default
logging.getLogger("OpenTLDR").setLevel(logging.INFO)   # More output
#logging.getLogger("OpenTLDR").setLevel(logging.DEBUG)  # So much output

In [ ]:
from opentldr import KnowledgeGraph
kg=KnowledgeGraph()

from opentldr.Domain import User, Request, Entity

## Parameters
OpenTLDR workflows use the notebook block tagged as "parameters" to inject variables (for example to configure special requests).

> **Do Not Change Variable Names in the Parameters Block** you are welcome to change the values of these parameter variables, but please do not change their names. They are used elsewhere in the notebook and in other workflow processes.

In [ ]:
# Workflow Parameters
request_date_repo_config = {'repo_type': 'files', 'path': './sample_data/request'}

# Local Parameters
spacy_model="en_core_web_lg"

## Named Entity Recognition


### Setup spaCy for NLP/NER

In [ ]:
import spacy
import sys
import subprocess

# if you have a GPU and your imstalled the spacy[cuda] package, it will use the GPU
spacy.prefer_gpu()

# SpaCy uses a language model that needs to be downloaded, this checks if that has been done
# and if it has not, it will download the model (and some dependencies) which can take a bit.
if not spacy.util.is_package(spacy_model):
        print("Downloading spaCy NLP Model...")
        #equivelent to running -> !{sys.executable} -m spacy download {spacy_model}
        subprocess.check_call([sys.executable, "-m", "spacy", "download", spacy_model])
else:
        print("spaCy model ({model}) is already downloaded.".format(model=spacy_model))

nlp = spacy.load(spacy_model)

### Function for running NER on a text string
The call to "spacy.display.render" prints out the text with annotations.

In [ ]:
def named_entity_recognition(text:str):
        doc = nlp(text)
        spacy.displacy.render(doc, style='ent')
        return doc.ents

## Analyst Query Specification

In [ ]:
from opentldr import DataRepo

# if you plan to only run this notebook multiple times you could clean out the content nodes each time
#kg.delete_all_requests()

if request_date_repo_config is not None:
    repo = DataRepo(kg, request_date_repo_config)
    list_of_uids =  repo.importData()
    print("Loaded {count} requests nodes from the repository.".format(count=len(list_of_uids)))

In [ ]:
for request_node in kg.get_all_requests():
    print("\nProcessing {title}:".format(title=request_node.title))

    # avoid adding duplicate entities for the same text value
    existing_entities=kg.get_entities_by_request(request_node)
    unique=[ e.text for e in existing_entities ]

    # perform NER on the entities in the request
    for entity in named_entity_recognition(request_node.text):
        if entity.label_ not in ['DATE','TIME','MONEY','ORDINAL','PERCENT','QUANTITY']:
            if entity.text not in unique:
                entity_node = kg.add_entity(content=request_node, text=entity.text, type=entity.label_)
                print(" - Added entity '{text}' of type {type}".format(text=entity_node.text, type=entity_node.type))
                unique.append(entity_node.text)


## Verify that we got Requests into the KG


In [ ]:
all_requests = kg.get_all_requests()
print("Found {count} request nodes in the knowledge graph:".format(count=len(all_requests)))

# Iterate thru the Request Nodes and print info for each
for request in all_requests:
    user=request.get_requested_by()
    print(" - {name} requested '{text}'".format(
        name=user.name, text=request.text))

# Close the KG

In [ ]:
kg.close()